In [1]:
# !pip install gym

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import gym
from gym_hnef import hnef_game, hnef_vars
from gym_hnef.envs import hnef_env
import model
import mcts
import agent
import config
import funcs
import memory

In [3]:
# 9 * 9 * 9 * 9, size of the action space
action_size = 6561
# initial state of the game
init_state = hnef_game.init_state('historical')

In [4]:
# Create the neural networks
current_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)
best_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)

In [5]:
current_NN.model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 5, 9, 9)]    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 9, 9)     2880        main_input[0][0]                 
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 9, 9)     256         conv2d[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64, 9, 9)     0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [6]:
# Create the players
current_player = agent.Agent('current_player', current_NN, init_state.shape, action_size)
best_player = agent.Agent('best_player', best_NN, init_state.shape, action_size)

In [7]:
iteration = 0

max_iter = 1e2

mem = memory.Memory(config.MEMORY_SIZE)

version = 0

while iteration < max_iter:
    print('Iteration number:', iteration)

    # Self play
    scores, mem = funcs.play_matches(best_player, best_player, mem=mem)
    print(scores)

    mem.clear_stmemory()

    if len(mem.ltmemory) >= config.MEMORY_SIZE:

        current_player.replay(mem.ltmemory)

        scores, _ = play_matches(best_player, current_player)
        print(scores)

        if scores['current_player'] > scores['best_player'] * config.SCORING_THRESHOLD:
            version += 1
            print(version)
            best_NN.model.set_weights(current_NN.model.get_weights())
    else:
        print('Memory size', str(len(mem.ltmemory)))
    iteration += 1

Iteration number: 0


ValueError: in user code:

    C:\Users\Notandi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\Notandi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Notandi\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Notandi\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Notandi\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Notandi\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:14